# M2 Agentic AI - 用反思改进 SQL 生成

## 1. 引言
### 1.1. 实验概览

在本实验中，你将探索反思模式如何改进一个将自然语言问题转换为 SQL 数据库查询的智能体工作流。你将看到智能体如何发现自身输出中的问题、进行修正，并在给出最终答案前提升响应质量。

### 🎯 1.2 学习目标

你将练习应用反思模式，以提升智能体工作流编写 SQL 查询的能力。为此，你将把一个反思步骤编码到工作流中，使得智能体能够：

* 审查自身的中间结果（如 SQL 草稿或工具输出）。
* 识别错误与缺漏。
* 检查其响应与工具使用。
* 在提交最终答案前对输出进行优化。

## 2. 环境设置：初始化环境与客户端

在本步骤中，你将准备好工作空间以便立即开始编码。你将：

1. **导入核心 Python 库**

   * `json`：处理结构化数据。
   * `pandas`：处理表格数据。
   * `dotenv`：加载环境变量（如 API 密钥）。

2. **加载环境变量**
   以确保你的工作空间已正确配置所需的密钥与设置。

3. **导入 `utils.py` 模块**
   其中包含用于格式化输出和支持后续工作流步骤的辅助函数。

**注意：** 如需查看 `utils.py` 的内容，可在顶部菜单选择 **File > Open**。


In [1]:
import json
import utils
import pandas as pd
from dotenv import load_dotenv

_ = load_dotenv()

### 2.1 开始使用 AISuite

本课程的实验将使用名为 aisuite 的包（[aisuite 仓库](https://github.com/andrewyng/aisuite)），它让你能够方便地调用由不同提供商托管的 LLM。Andrew 将在模块 3 中进一步介绍 aisuite。现在初始化 `aisuite client`。该客户端为你提供统一的方式连接并与不同 LLM 交互——无需担心每个模型有各自的初始化方式。

In [2]:
import aisuite as ai

client = ai.Client()

### 2.2. 设置数据库

在此步骤中，你将创建一个名为 **`products.db`** 的本地 SQLite 数据库。
数据库将自动填充随机生成的产品数据。

你将在实验的后续步骤中使用这些数据练习与测试 SQL 查询。

In [3]:
utils.create_transactions_db()

已创建 SQLite 数据库 'products.db'，包含一个事件溯源的 'transactions' 表。


你可以运行下方单元以查看数据表的架构（schema）。

In [4]:
utils.print_html(utils.get_schema('products.db'))

在该表中，每一行都代表一个**事件**（插入、补货、销售或价格更新）。关于库存水平、销售额或价格趋势等分析都可由这些事件聚合得到。

<div style="border:1px solid #93c5fd; border-left:6px solid #3b82f6; background:#eff6ff; border-radius:6px; padding:12px 14px; color:#1e3a8a; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">  
<strong>🔎 架构概览：</strong><br><br>  
• <code>id</code> → 事件的唯一 ID（自增）。<br>  
• <code>product_id</code>、<code>product_name</code>、<code>brand</code>、<code>category</code>、<code>color</code> → 用于标识产品。<br>  
• <code>action</code> → 事件类型（<em>insert</em>、<em>restock</em>、<em>sale</em>、<em>price_update</em>）。<br>  
• <code>qty_delta</code> → 库存变化（插入/补货为 +，销售为 –，价格更新为 0）。<br>  
• <code>unit_price</code> → 当时价格（补货为 NULL）。<br>  
• <code>notes</code> → 事件的可选描述。<br>  
• <code>ts</code> → 记录事件时的时间戳。<br>  
</div>  

> 基于该架构，你可以通过聚合事件历史随时重构**当前状态**（库存、最新价格、总销售额）。

## 3. 构建 SQL 生成器

### 3.1. 使用 LLM 查询数据库

在本步骤中，你将使用一个函数把自然语言问题转换为 SQL 查询。

你将把问题与数据库架构作为输入提供给模型；随后 LLM 会生成用于回答你问题的 SQL 查询。

通过这种方式，**你**可以专注于提出问题，而由模型负责撰写查询语句。


In [5]:
def generate_sql(question: str, schema: str, model: str) -> str:
    prompt = f"""
    你是一名 SQL 助理。根据给定的数据库架构与用户问题，编写适用于 SQLite 的 SQL 查询。

    架构：
    {schema}

    用户问题：
    {question}

    仅返回 SQL 语句。
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )
    return response.choices[0].message.content.strip()

运行下方单元，查看 **`generate_sql`** 如何基于自然语言问题为 `transactions` 表生成 **第一版（V1）** SQL 查询。
只需提供**问题**、**架构（schema）**与**模型名**，函数将返回初始查询草稿。

In [6]:
# 示例：使用 generate_sql

# 我们将架构以字符串形式提供
schema = """
表名: transactions
id (INTEGER)
product_id (INTEGER)
product_name (TEXT)
brand (TEXT)
category (TEXT)
color (TEXT)
action (TEXT)
qty_delta (INTEGER)
unit_price (REAL)
notes (TEXT)
ts (DATETIME)
"""

# 我们用自然语言提出关于数据的问题
question = "哪种颜色的产品总销售额最高？"

utils.print_html(question, title="用户问题")

# 使用指定模型生成 SQL 查询
sql_V1 = generate_sql(question, schema, model="openai:gpt-4.1")

# 展示生成的 SQL 查询
utils.print_html(sql_V1, title="SQL 查询 V1")


#### 3.1.1. 查询执行

现在你将执行**第一版（V1）SQL 查询**并检查其结果。此步骤很重要，因为它能帮你验证 LLM 生成的查询是否确实从数据库检索到了你期望的信息。  

- **`utils.execute_sql(...)`**：针对 `products.db` 执行生成的 V1 查询，并以 pandas DataFrame 返回输出。使用 DataFrame 便于检查、分析，并将结果传递到后续步骤。  

- **`utils.print_html(...)`**：将 DataFrame 渲染为整洁的 HTML 表格，使原始输出更易读，便于快速判断查询结果是否契合用户问题。  


In [7]:
# 针对 products.db 执行生成的 SQL 查询（sql_V1）。
# 结果将以 pandas DataFrame 返回。
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')

# 在笔记本中把 DataFrame 渲染为 HTML 表格。
# 这样可使查询输出更易于阅读与理解。
utils.print_html(df_sql_V1, title="SQL 查询 V1 的输出 - ❌ 未能完整回答问题")


color,total_sales
blue,-190571.46


<div style="border:1px solid #ef4444; border-left:6px solid #dc2626; background:#fee2e2; border-radius:6px; padding:12px 14px; color:#7f1d1d; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

❌ <strong>V1 输出问题：</strong> 查询结果未能完整回答问题，因为总销售额为<em>负值</em>（`-190571.46`）。  
<br><br>
在该数据集中，<code>qty_delta</code> 在**销售**时记录为**负数**（库存减少），而在**退货或补货**时记录为**正数**（库存增加）。查询使用了 <code>SUM(qty_delta)</code>，因此在跨交易求和时，负数占主导导致总和为负。  
<br>
这意味着该 SQL 在技术上是有效的，但在语义上不正确——总销售额不应以负值表示。  
<br>
➡️ 该问题<strong>进一步表明反思的必要性</strong>：模型需要优化其逻辑（例如在计算销售额时将 <code>qty_delta</code> 乘以 <code>-1</code>，或使用 <code>ABS()</code>），使最终查询能够反映真实的销售总额。  

</div>


### 3.2. 通过反思改进 SQL 查询

在本节中，你将学习如何通过**反思**来改进 SQL 查询。

首先，LLM 可仅根据问题与架构审查**SQL 文本**，在需要时给出改进建议。
随后，LLM 还可考虑**实际查询执行结果**，以捕捉负值总计、缺失筛选或分组错误等细微问题。

综合上述方法，反思可以让你的 SQL 工作流更**可靠且准确**——先在“纸面上”检查逻辑，再用真实数据进行验证。

#### 3.2.1. 初次尝试：改进 SQL 查询

在此函数中，你让 LLM 根据原始问题与架构（如 3.1 节定义的架构）**审查**一条 SQL 查询。模型会反思该查询是否完整回答了问题；若没有，则提出改进版本。

* **输入**：

  * 用户的**问题**
  * **原始 SQL 查询**
  * **表架构（schema）**

* **输出**：

  * **feedback** → 简短评估（例如：“有效但缺少日期筛选”）
  * **refined_sql** → 最终 SQL（若原查询正确则不变，若需改进则更新）

此函数**不执行 SQL**。它仅检查查询并在逻辑与意图不完全匹配时给出优化建议。

In [8]:
def refine_sql(
    question: str,
    sql_query: str,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    反思查询的“展示输出”是否回答了用户问题，
    如有需要，提出改进版 SQL。
    返回 (feedback, refined_sql)。
    """
    prompt = f"""
你是一位 SQL 审查与优化专家。

用户问题：
{question}

原始 SQL：
{sql_query}

表架构：
{schema}

步骤 1：简要评估 SQL 输出是否完整回答用户问题。
步骤 2：若需要改进，请提供适用于 SQLite 的优化版 SQL 查询。
若原始 SQL 已正确，请保持不变返回。

严格返回仅包含以下两个字段的 JSON：
{{
  "feedback": "<1-3 句解释问题或确认正确性>",
  "refined_sql": "<final SQL to run>"
}}
"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
    )

    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # 若模型未返回有效 JSON 的回退处理
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql


运行下方单元以生成**改进后的 SQL 查询（V2）**。该步骤将：

* 显示在 3.1 节针对问题“哪种颜色的产品总销售额最高？”生成的**初始 SQL 查询（V1）**
* 展示模型的**反馈**及其**改进版 SQL（V2）**
* 针对数据库执行**原始 SQL（V1）**并呈现其**实际输出**，帮助你了解为何需要改进。

In [9]:
# 示例：改进生成的 SQL（V1 → V2）

feedback, sql_V2 = refine_sql(
    question=question,
    sql_query=sql_V1,   # <- 来自 generate_sql()（V1）
    schema=schema, # <- 复用 3.1 节的架构
    model="openai:gpt-4.1"
)

# 展示原始问题
utils.print_html(question, title="用户问题")

# --- V1 ---
utils.print_html(sql_V1, title="生成的 SQL 查询（V1）")

# 执行并展示 V1 输出
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')
utils.print_html(df_sql_V1, title="SQL 查询 V1 的输出 - ❌ 未能完整回答问题")

# --- 反馈 + V2 ---
utils.print_html(feedback, title="对 V1 的反馈")
utils.print_html(sql_V2, title="改进后的 SQL 查询（V2）")

# 执行并展示 V2 输出
df_sql_V2 = utils.execute_sql(sql_V2, db_path='products.db')
utils.print_html(df_sql_V2, title="SQL 查询 V2 的输出 - ❌ 未能完整回答问题")

color,total_sales
blue,-190571.46


color,total_sales
blue,-190571.46


<div style="border:1px solid #fecaca; border-left:6px solid #dc2626; background:#fee2e2; border-radius:6px; padding:12px 14px; color:#7f1d1d; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

❌ **如你所见…**  
尽管生成的 SQL 看起来是正确的且反馈也予以确认，但**实际的查询结果**显示 `total_sales` 为负值。  

如前所述，这发生在 SQL 直接计算 `qty_delta * unit_price` 而未考虑销售事件会记录**负数数量**（`qty_delta < 0`）的情况下。正负号的反转是一个细微的语义问题，单靠审查查询文本并不总能发现。  

👉 因此智能体还必须对**执行输出**进行反思——以捕捉诸如符号错误、缺失的筛选或错误的聚合等问题。来自查询结果的外部反馈使改进过程扎根于真实数据，而不仅仅是 SQL 的结构。  

</div>

#### 3.2.2. 最终方法：结合外部反馈改进 SQL 查询

不同于上一步仅**审查 SQL 文本**，现在你将提供**实际的查询执行结果**作为外部反馈。  
这些反馈来自在数据库上运行 SQL 查询——就像 Andrew 的视频示例那样——因此 LLM 能使用真实输出来评估查询是否真正回答了问题。

在此步骤中，你将获得：

* 基于**真实执行输出**的简短评估  
* 具体的改进建议（例如缺失筛选、分组问题、符号错误）  
* 更贴合原始问题的改进版 SQL 语句  

In [10]:
def refine_sql_external_feedback(
    question: str,
    sql_query: str,
    df_feedback: pd.DataFrame,
    schema: str,
    model: str,
) -> tuple[str, str]:
    """
    评估 SQL 结果是否回答用户问题；如有必要，提出改进版查询。
    返回 (feedback, refined_sql)。
    """
    prompt = f"""
    你是一位 SQL 审查与优化专家。

    用户问题：
    {question}

    原始 SQL：
    {sql_query}

    SQL 输出：
    {df_feedback.to_markdown(index=False)}

    表架构：
    {schema}

    步骤 1：简要评估该 SQL 输出是否回答了用户问题。
    步骤 2：若可改进，请提供优化后的 SQL 查询。
    若原始 SQL 已正确，请保持不变返回。

    请严格返回仅包含以下两个字段的 JSON 对象：
    - "feedback": 简短评估与建议
    - "refined_sql": 需要执行的最终 SQL
    """

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
    )

    
    content = response.choices[0].message.content
    try:
        obj = json.loads(content)
        feedback = str(obj.get("feedback", "")).strip()
        refined_sql = str(obj.get("refined_sql", sql_query)).strip()
        if not refined_sql:
            refined_sql = sql_query
    except Exception:
        # 若模型未返回有效 JSON 的回退处理：
        # 使用原始内容作为反馈并保留原始 SQL
        feedback = content.strip()
        refined_sql = sql_query

    return feedback, refined_sql

运行下方单元，观察如何利用查询结果的**外部反馈**来改进 SQL。

在此示例中你将：

* 展示基于你的问题生成的**原始 SQL 查询（V1）**。
* 展示 **V1 的输出**，说明为何初版未能完整回答问题。
* 基于该输出提供 LLM 的**反馈**。
* 给出能够解决问题的**改进版 SQL（V2）**。
* 执行 **V2** 并展示其输出，确认它已 ✅ 完整回答问题。

In [11]:
# 示例：结合外部反馈改进 SQL（V1 → V2）

# 执行原始 SQL（V1）
df_sql_V1 = utils.execute_sql(sql_V1, db_path='products.db')

# 使用外部反馈进行评估与改进
feedback, sql_V2 = refine_sql_external_feedback(
    question=question,
    sql_query=sql_V1,   # V1 查询
    df_feedback=df_sql_V1,    # V1 的输出
    schema=schema,
    model="openai:gpt-4.1"
)

# --- V1 ---
utils.print_html(question, title="用户问题")
utils.print_html(sql_V1, title="生成的 SQL 查询（V1）")
utils.print_html(df_sql_V1, title="SQL 查询 V1 的输出 - ❌ 未能完整回答问题")

# --- 反馈与 V2 ---
utils.print_html(feedback, title="对 V1 的反馈")
utils.print_html(sql_V2, title="改进后的 SQL 查询（V2）")

# 执行并展示 V2 结果
df_sql_V2 = utils.execute_sql(sql_V2, db_path='products.db')
utils.print_html(df_sql_V2, title="SQL 查询 V2（结合外部反馈）的输出 - ✅ 已完整回答问题")


color,total_sales
blue,-190571.46


color,total_sales
white,358315.09


<div style="border:1px solid #bbf7d0; border-left:6px solid #22c55e; background:#dcfce7; border-radius:6px; padding:12px 14px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

✅ **成功！**  
通过改进后的 SQL，负值 `qty_delta` 问题已用 `ABS(qty_delta)`（或对销售等价地使用 `-qty_delta`）修正。  

现在输出为**正且更有意义**，显示了总销售额最高的正确颜色。这体现了结合**反思 + 外部反馈**的重要性：  
- 仅看 SQL 文本似乎没有问题。  
- 实际执行输出揭示了符号错误。  
- 改进后的查询修正了逻辑，得到正确答案。  

</div>


### 3.3. 整体串联 — 构建数据库查询工作流

在此步骤中，**你**将使用一个函数，将创建、运行与改进 SQL 查询的整个流程自动化。

该工作流包括以下关键步骤：

1. **提取数据库架构（schema）**
2. 从自然语言问题**生成初始（V1）SQL 查询**
3. **结合执行反馈反思 V1** —— 审阅实际查询结果并在需要时改进 SQL
4. **执行改进后的（V2）SQL 查询**，确保它完整回答你的问题

最终，**你**将看到：

* 初始与改进后的查询
* 各自的输出
* LLM 提供的解释改进的反馈

这让**你**能够更顺畅、准确且全自动地处理 SQL 查询。

In [ ]:
def run_sql_workflow(
    db_path: str,
    question: str,
    model_generation: str = "openai:gpt-4.1",
    model_evaluation: str = "openai:gpt-4.1",
):
    """
    端到端工作流：用于生成、执行、评估并改进 SQL 查询。

    步骤：
      1) 提取数据库架构
      2) 生成 SQL（V1）
      3) 执行 V1 → 展示输出
      4) 结合执行反馈反思 V1 → 提出改进版 SQL（V2）
      5) 执行 V2 → 展示最终答案
    """

    # 1) 结构
    schema = utils.get_schema(db_path)
    utils.print_html(
        schema,
        title="📘 第 1 步 — 提取数据库架构"
    )

    # 2) 生成 SQL（V1）
    sql_v1 = generate_sql(question, schema, model_generation)
    utils.print_html(
        sql_v1,
        title="🧠 第 2 步 — 生成 SQL（V1）"
    )

    # 3) 执行 V1 
    df_v1 = utils.execute_sql(sql_v1, db_path)
    utils.print_html(
        df_v1,
        title="🧪 第 3 步 — 执行 V1（SQL 输出）"
    )

    # 4) 结合执行反馈反思 V1 → 提出改进版 SQL（V2）
    feedback, sql_v2 = refine_sql_external_feedback(
        question=question,
        sql_query=sql_v1,
        df_feedback=df_v1,          # 外部：V1 的执行结果
        schema=schema,
        model=model_evaluation,
    )
    utils.print_html(
        feedback,
        title="🧭 第 4 步 — 反思 V1（反馈）"
    )
    utils.print_html(
        sql_v2,
        title="🔁 第 4 步 — 改进后的 SQL（V2）"
    )

    # 5) 执行 V2 → 展示最终答案
    df_v2 = utils.execute_sql(sql_v2, db_path)
    utils.print_html(
        df_v2,
        title="✅ 第 5 步 — 执行 V2（最终答案）"
    )


### 3.4. 运行 SQL 工作流

现在轮到**你**来执行完整的 SQL 处理流水线。你可以尝试以下 OpenAI 模型的不同组合，它们的能力与性能各不相同：

* `openai:gpt-4o`
* `openai:gpt-4.1`
* `openai:gpt-4.1-mini`
* `openai:gpt-3.5-turbo`

💡 在此工作流中，`openai:gpt-4.1` 往往在自我反思任务上表现最佳。

**重要说明：** 由于大语言模型（LLM）具有随机性，每次运行可能会返回略有不同的结果。
建议你尝试不同模型与组合，以找到最适合**你**的设置。

In [13]:
run_sql_workflow(
    "products.db", 
    "哪种颜色的产品总销售额最高？",
    model_generation="openai:gpt-4.1",
    model_evaluation="openai:gpt-4.1"
)

color,total_sales
blue,-190571.46


color,total_sales
white,358315.09


## 4. 总结要点

完成本实验后，**你**学会了：

* 使用 LLM 将自然语言问题转换为 SQL 查询  
* 应用**反思模式**（包含/不包含外部反馈）改进生成的 SQL  
* 自动化完整的 SQL 工作流：从架构提取到查询执行与改进  
* 尝试不同的 LLM 模型，以比较性能与准确性  

关键洞察在于：反思让**你的**智能体更可靠。智能体不再停留在第一次尝试，而是能够审查、改进，并给出更贴合**你的**意图的结果。


<div style="border:1px solid #22c55e; border-left:6px solid #16a34a; background:#dcfce7; border-radius:6px; padding:14px 16px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

🎉 <strong>恭喜！</strong>  

你已完成关于构建**具备智能体特性的 SQL 工作流（V1 → V2）**的实验。  

在此过程中，**你**练习了如何将反思、执行与验证组合成一个可靠的管线。
你也见识了**外部反馈**的重要性：有时仅看 SQL 文本似乎正确，但实际执行输出会暴露隐藏问题（如负数总计、缺失筛选或分组错误）。  

掌握这些技能后，**你**已准备好设计自己的**智能体管线**——即：  
* 生成初始查询（V1）  
* 结合执行反馈进行反思与改进（V2）  
* 交付更安全、更透明且更值得信赖的结果 🌟  

</div>
